# Learn OpenAI Whisper - Chapter 7

## Notebook 3: Building a real-time ASR demo with Hugging Face Whisper

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1G1KpicRfLpa88jPXrb_0nIXFueRq7MOV)

In this notebook, we will leverage the power of Gradio, a user interface library, to rapidly construct an interactive demo of the Whisper model. This demo will allow you or others to test the model's performance by speaking into the microphone on your device. The notebook is structured into three main sections:

1. **Setting up the Automatic Speech Recognition (ASR) model using the Hugging Face Transformers library**: We will load and configure the necessary components from the transformers library to prepare the ASR model for our demo.

2. **Creating a full-context ASR demo**: In this section, we will build a demo where the user speaks the entire audio before the ASR model processes it and generates the transcription.

3. **Creating a streaming ASR demo**: We will extend the previous demo to support real-time streaming, allowing the ASR model to transcribe the audio as the user speaks, providing a more interactive experience.

By the end of this notebook, you will have a solid understanding of how to create engaging demos for speech recognition models using Gradio and the Hugging Face Transformers library.


## Preparing the Development Environment

Before diving into building the speech recognition demos, it's crucial to set up our development environment with the necessary dependencies. In this section, we will:

1. Install the required libraries, such as Gradio, to ensure a smooth development process.
2. Configure the environment to work seamlessly with the Hugging Face Transformers library, allowing us to leverage pre-trained models and powerful NLP tools.

By properly setting up our environment, we lay the foundation for an efficient and hassle-free coding experience throughout the notebook.


In [ ]:
%%capture
!pip -q install gradio

**Attention: Setting Up Your Hugging Face Token**

To ensure a seamless experience while working with this notebook, it is essential to set up your Hugging Face token. The notebook will be loading transformer classes and models from the Hugging Face repository, which requires authentication using a valid token.

If you haven't created a Hugging Face token yet or need a refresher on the process, please refer to the [notebook for Chapter 3](https://github.com/PacktPublishing/Learn-OpenAI-Whisper/blob/main/Chapter03/LOAIW_ch03_working_with_audio_data_via_Hugging_Face.ipynb). This resource provides step-by-step instructions on how to create and configure your Hugging Face token.

By setting up your token correctly, you'll be able to access the full range of features and models available in the Hugging Face ecosystem, enabling you to build powerful speech recognition demos with ease.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import whoami

whoami()
# you should see something like {'type': 'user',  'id': '...',  'name': 'Wauplin', ...}

## Step 1: Loading the Transformers ASR Model

To begin building our speech recognition demo, we first need to have an Automatic Speech Recognition (ASR) model. You can either train your own model or use a pre-trained one. In this tutorial, we will leverage a pre-trained ASR model called "whisper" from OpenAI.

Loading the "whisper" model from the Hugging Face Transformers library is a straightforward process. Here's the code snippet to accomplish this:

```python
from transformers import pipeline
p = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")
```

With just these two lines of code, we initialize a pipeline for automatic speech recognition using the "openai/whisper-base.en" model. The pipeline abstracts away the complexities of working with the model directly, providing a high-level interface for performing ASR tasks.

By utilizing a pre-trained model like "whisper", we can quickly get started with building our demo without the need for extensive model training. This allows us to focus on integrating the model into our application and creating an engaging user experience.

## Step 2: Building a Full-Context ASR Demo with Transformers

Our first step in creating the speech recognition demo is to build a *full-context* ASR demo. In this demo, the user will speak the entire audio before the ASR model processes it and generates the transcription. Thanks to Gradio's intuitive interface, building this demo is a breeze.

We'll start by creating a function that wraps around the `pipeline` object we initialized earlier. This function will serve as the core of our demo, handling the audio input and generating the transcription.

To capture the user's audio input, we'll utilize Gradio's built-in `Audio` component. This component will be configured to accept input from the user's microphone and return the filepath of the recorded audio. For displaying the transcribed text, we'll use a simple `Textbox` component.

The `transcribe` function, which is the heart of our demo, takes a single parameter called `audio`. This parameter represents the audio data recorded by the user, stored as a NumPy array. However, the `pipeline` object expects the audio data to be in the `float32` format. To ensure compatibility, we first convert the audio data to `float32` and then normalize it by dividing it by its maximum absolute value. Finally, we pass the processed audio data to the `pipeline` object to obtain the transcribed text.

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np

transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    return transcriber({"sampling_rate": sr, "raw": y})["text"]


demo = gr.Interface(
    transcribe,
    gr.Audio(sources=["microphone"]),
    "text",
)

demo.launch(debug=True)

## Step 3: Enhancing the Demo with Real-time Streaming Capabilities

To create a streaming ASR demo, we need to make the following changes:

1. Set **streaming=True** in the **Audio** component to enable continuous audio capture from the user's microphone.
2. Set **live=True** in the **Interface** to ensure that the interface updates dynamically as new audio data is received.
3. Add a **state** variable to the interface to store the recorded audio and the previous transcription.

In the streaming demo, we use a state variable to keep track of the audio history and the previous transcription. The **transcribe** function is called whenever a new small chunk of audio is received, and it needs to process the new chunk along with the previously recorded audio.

To improve the accuracy and coherence of the transcription, we introduce a dynamic window size based on the duration of the new audio chunk and a small overlap between consecutive windows. Here's how the **transcribe** function works:

1. If the **state** is None, initialize an empty numpy array (**stream**) to store the audio and an empty string (**previous_text**) to store the previous transcription.
2. Extract the sampling rate (**sr**) and audio data (**y**) from the **new_chunk**.
3. Calculate the duration of the new audio chunk and normalize the audio data.
4. Introduce an overlap of half a second between consecutive windows to ensure continuity in the transcription.
5. Concatenate the new audio chunk to the existing **stream**, taking into account the overlap.
6. Transcribe the entire **stream** using the **transcriber** object.
7. Update the **previous_text** by removing the overlap from the end of the previous transcription and concatenating it with the new transcription.
8. Return the updated **stream** and **combined_text** as the state, and the **combined_text** as the transcription output.

By using a dynamic window size and introducing an overlap between consecutive windows, we can improve the accuracy and coherence of the streaming transcription. The small overlap helps maintain continuity in the transcription and reduces the occurrence of overlapping or missing words.

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np

transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(state, new_chunk):
    if state is None:
        stream = np.array([], dtype=np.float32)
        previous_text = ""
    else:
        stream, previous_text = state

    sr, y = new_chunk
    duration = len(y) / sr
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    overlap = int(sr * 0.5)  # Half a second overlap
    if len(stream) > 0:
        stream = np.concatenate([stream[-overlap:], y])
    else:
        stream = y

    # Transcribe the current chunk
    new_text = transcriber({"sampling_rate": sr, "raw": stream})["text"]

    # Update the previous text based on the overlap
    if len(previous_text) > 0:
        overlap_text = previous_text[-int(len(previous_text) * 0.1):]  # Last 10% of previous text
        combined_text = previous_text[:-len(overlap_text)] + new_text
    else:
        combined_text = new_text

    return (stream, combined_text), combined_text

demo = gr.Interface(
    transcribe,
    ["state", gr.Audio(sources=["microphone"], streaming=True)],
    ["state", "text"],
    live=True,
)

demo.launch(debug=True)